In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
# hyperparameters

batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
head_size = n_embd // n_head
n_layer = 6
dropout = 0.2

In [ ]:
torch.manual_seed(1337)
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

--2024-09-25 16:05:03--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2024-09-25 16:05:03 (29.5 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [ ]:
# evaluation while training

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters, device=device)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
class FeedForwardLayer(nn.Module):

    def __init__(self):
        super().__init__()
        self.ffwd = nn.Linear(n_embd, 4 * n_embd)
        self.relu = nn.ReLU()
        self.proj = nn.Linear(4 * n_embd, n_embd)

    def forward(self, x):
        x = self.ffwd(x)
        x = self.relu(x)
        out = self.proj(x)
        return out


class SelfAttentionHead(nn.Module):

    def __init__(self):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size)
        self.query = nn.Linear(n_embd, head_size)
        self.value = nn.Linear(n_embd, head_size)
        self.register_buffer('tril', torch.tril(torch.ones((block_size, block_size))))

    def forward(self, x):
        B, T, C = x.shape

        k = self.key(x)
        q = self.query(x)

        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)

        v = self.value(x)
        out = wei @ v

        return out


class MultiHeadAttention(nn.Module):

    def __init__(self):
        super().__init__()
        self.heads = nn.ModuleList([SelfAttentionHead() for i in range(n_head)])
        self.proj = nn.Linear(n_embd, n_embd)

    def forward(self, x):
        x = torch.cat([head(x) for head in self.heads], dim=-1)
        out = self.proj(x)
        return out


class AttentionBlock(nn.Module):

    def __init__(self):
        super().__init__()
        self.mha_attn = MultiHeadAttention()
        self.ffwd = FeedForwardLayer()
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.mha_attn(self.ln1(x))  # communication
        out = x + self.ffwd(self.ln2(x))    # computation
        return out


class CausalLM(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.attn_block = AttentionBlock()
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, inputs, targets=None):
        B, T = inputs.shape

        tok_emb = self.token_embedding_table(inputs) # B, T, C
        pos_emb = self.position_embedding_table(torch.arange(inputs.shape[1], device=device)) # B, T, C
        x = tok_emb + pos_emb # B, T, C
        x = self.attn_block(x) # B, T, C

        logits = self.lm_head(x) # B, T, V
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, vocab_size), targets.view(-1,))
        return logits, loss

    @torch.no_grad()
    def generate(self, x, max_new_tokens=100):
        for i in range(max_new_tokens):

            x_block_size = x[:, -block_size:]

            logits, loss = self(x_block_size)
            probs = F.softmax(logits[:, -1, :], dim=-1)

            next_idxs = torch.multinomial(probs, num_samples=1)
            x = torch.cat([x, next_idxs], dim=-1)
        return x

In [ ]:
# model initialization

model = CausalLM().to(device)

In [ ]:
# optimizer initialization

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
# training

for i in range(max_iters):     # number of training steps

    if i % eval_interval == 0 or i == max_iters - 1:
        losses = estimate_loss()
        print(f"step {i}: train loss {losses['train']:.4f} val loss {losses['val']:.4f}")

    x, y = get_batch('train')  # custom function to get the training batch
    logits, loss = model(x, y)     # forward pass

    optimizer.zero_grad()      # to clear out the previous gradients
    loss.backward()            # backward pass, calculating gradients of parameters wrt loss
    optimizer.step()           # updating parameters with gradients

step 0: train loss 4.4595 val loss 4.4697
step 500: train loss 2.1611 val loss 2.2096
step 1000: train loss 1.8178 val loss 1.9473
step 1500: train loss 1.6365 val loss 1.8156
step 2000: train loss 1.5444 val loss 1.7466
step 2500: train loss 1.4768 val loss 1.6920
step 3000: train loss 1.4348 val loss 1.6643
step 3500: train loss 1.3986 val loss 1.6403
step 4000: train loss 1.3732 val loss 1.6194
step 4500: train loss 1.3540 val loss 1.6167
step 4999: train loss 1.3352 val loss 1.6032


In [ ]:
# inference

xx = torch.zeros((2, 1), dtype=torch.long, device=device)
yy = model.generate(xx, 500)
print(decode(yy[1].tolist()))


I are you, and I take the crown tell bedced
'Gainst of York, out
Myself dready and charge buried to reasoning
Which is not the trums in the jestice.

Nurse:
Well the kinst thou docers, to Yorked from god
I executest vengeed that I under manner where?
When for some to smed thus?

CATEN:
My Lord of loving King of the nurse to beard thy cares foul approachments,
I wink our honest a any did inkling else.
To they stratege for this,
That's departiles,
And suburdenititufuly forfeignal we grave;
And tel
